In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math

from numpy.random import default_rng
rng = default_rng()
import random
random.seed(1000)

from mpl_toolkits.mplot3d import Axes3D
from time import time

# Set data type
#DTYPE='float32'
DTYPE='float64'
tf.keras.backend.set_floatx(DTYPE)
print('TensorFlow version used: {}'.format(tf.__version__))

plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
# Cost of non-smooth control
eps = 0.01

In [ ]:
def noise(num_sample, N, k, dt):
    dW = np.random.normal(loc=0.0, scale=np.sqrt(dt), size=(num_sample, N,k)).astype(DTYPE) 
    return dW

The state variable $(X_t)_{t \in \{0,1,\dots, N-1 \}} =(B_t, P_t)_{t \in \{0,1,\dots, N-1 \}}$ where $B_t$ is the battery level and $P_t$ is the (spot) market price. We can assume that $P_t \in \mathbb{R}^d$ and $P_t^0$ is the current price. In the following cell below, we simulate the price process using an Ornstein-Uhlenbeck process.

In [ ]:
num_days_price = 7

# Final time
T = num_days_price

# Dimension of the state space
dim = 2


# Steps per day
steps_per_day = 48

# Number of time steps
N = num_days_price*steps_per_day # Transforms in hours 7*24=168

# Derive time step size and t_space
dt = T/N
t_space = np.linspace(0., T, N + 1)

pi=math.pi
# f= lambda t : np.cos(2.*pi*t)
#f = lambda t : 5. * np.cos(2.*pi*t) + 16. + 2. * np.cos(6.*pi*t + 0.3) - 1. * np.cos(24*pi*t-0.4)
f = lambda t : (1. * np.cos(2.*pi*t) + 1. + 0.5 * np.cos(6.*pi*t + 0.3) )*0.25

# Define a price shift. Won't be calculated in optimised profit, but included for plots.
price_shift = 10.

In [ ]:
# p0_vec = np.array([0.1, -0.2, -0.4, 0.5, 0.7])
# sigmaOU_vec = np.array([6., 20.0, 3., 10., 20.])
# thetaOU_vec = np.array([10., 1.2, 2.4, 5., 0.3])

p0_vec = np.array([-0.1, +0.2,])
sigmaOU_vec = np.array([6., 12.0])
thetaOU_vec = np.array([10., 1.2])


k = np.size(p0_vec)
p0_vec = np.reshape(p0_vec,(k,1))
muOU_vec = np.zeros(k)

num_sample = 100
#num_sample = 10000
#num_sample = 5000

def simOU(p0_vec,tV,muOU_vec,sigmaOU_vec,thetaOU_vec,dW):
    X = np.zeros((np.shape(dW)[0],np.size(p0_vec),np.size(tV)))
    for j in range(np.size(p0_vec)):
        W = sigmaOU_vec[j]*dW[:,:,j]                
        X[:,j,0] = p0_vec[j,0]
        for i in range(len(tV)-1):
            X[:,j,i+1] = X[:,j,i]+thetaOU_vec[j]*(muOU_vec[j]-X[:,j,i])*(tV[i+1]-tV[i]) + np.sqrt(tV[i+1]-tV[i])*W[:,i]
    return X


price_vec = simOU(p0_vec,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,noise(num_sample, N, k, dt)) # (5, 101) 5 days
sum_price_vec_days= np.sum(price_vec, axis=1) 
#sum_price_vec_days=sum_price_vec_days.reshape(1,101)
# Forward curve
price_curve = f(t_space) + sum_price_vec_days+price_shift
plt.plot(t_space,price_curve[0],t_space,f(t_space)+price_shift);

In [ ]:
# Tidal component semi-diurnal table
periods = np.array([12.4206012, 12., 12.65834751, 23.93447213, 6.210300601, 25.81933871, 4.140200401, 8.177140247, 6., 6.269173724]) 
# M2 S2 N2 (semi-diurnal), K1 (diurnal), M4 (Short period) O1 (diurnal) M6 (short period) MK3 (short period) S4 (short period) MN4 (short period)
# In hours, by NOAA ranking

# Amplitudes (half of peak-to-peak) amplitude are given below
ME = [268.7, 42.0, 54.3, 15.6, 6.0, 11.9, 5.1, 0., 0., 2.3] # Amplitudes for  Eastport, Maine (ME)
# Here the last to second and the previous are set to zero (missing values in the table?)

AK = [97.3,32.5,20.1,39.8, 0.9,25.9,1.0,0.5,0.0,0.3] #Amplitudes for  Kodiak, Alaska (AK)
#  Here the last to second is set to zero (missing value in the table?)

amp = 0.1*np.array(ME)
no_terms = 8
phase_shift = rng.uniform(0.,1.,no_terms)*periods[:no_terms]

# print(phase_shift)

cutoff = 0.6
flow_max = 20.**(1/3)

#OLD
#days = 7
#N = 10000
#T = 24*days
#t = np.linspace(0,T,N)

num_days_power = 7    # This is the same as for the num_days_price
N_power = num_days_power*steps_per_day # This is the same as N for the price
t = np.linspace(0., num_days_power, N_power+1)

# plt.plot(t,np.dot(amp[:no_terms],np.sin(2*np.pi*(np.tile(t*24,(no_terms,1)).T-phase_shift[:no_terms])/periods[:no_terms]).T))
# plt.figure()
# plt.plot(t,np.dot(amp[:no_terms]/periods[:no_terms],np.cos(2*np.pi*(np.tile(t*24,(no_terms,1)).T-phase_shift[:no_terms])/periods[:no_terms]).T))
flow = np.dot(amp[:no_terms]/periods[:no_terms],np.cos(2*np.pi*(np.tile(t*24,(no_terms,1)).T-phase_shift[:no_terms])/periods[:no_terms]).T)
#dG_t =flow
dG_t =(np.minimum(np.abs(flow),flow_max)*(np.abs(flow)>cutoff))**3
#dG_t =np.abs(flow)
# plt.figure()
plt.plot(t,dG_t)
plt.title("Tidal Energy Generation");
#plt.plot(t,np.abs(flow))


#OLD
#plt.plot(t/24,np.dot(amp[:no_terms],np.sin(2*np.pi*(np.tile(t,(no_terms,1)).T-phase_shift[:no_terms])/periods[:no_terms]).T))
## np.tiles: Construct an array by repeating A the number of times given by reps.
#plt.figure()
#plt.plot(t/24,np.dot(amp[:no_terms]/periods[:no_terms],np.cos(2*np.pi*(np.tile(t,(no_terms,1)).T-phase_shift[:no_terms])/periods[:no_terms]).T))
#flow = np.dot(amp[:no_terms]/periods[:no_terms],np.cos(2*np.pi*(np.tile(t,(no_terms,1)).T-phase_shift[:no_terms])/periods[:no_terms]).T)
#plt.figure()
#plt.plot(t/24,np.abs(flow)**3)

In [ ]:
class BSDEModel(tf.keras.Model):
    def __init__(self, **kwargs):
        
        # Call initializer of tf.keras.Model
        super().__init__(**kwargs)
        
        # Initialize the control randomly
        #control_0 = np.random.uniform(-1e-1, 1e-1, size=(1, dim)).astype(DTYPE)
        control_0 = np.random.uniform(-1e-1, 1e-1, size=(1)).astype(DTYPE)    
        self.control_0 = tf.Variable(control_0)
        
        # Create template of dense layer without bias and activation
        _dense = lambda dim: tf.keras.layers.Dense(
            units=dim, # number of nodes
            activation=None,
            use_bias=False)
        
        # Create template of batch normalization layer
        _bn = lambda : tf.keras.layers.BatchNormalization(
            momentum=.99,
            epsilon=1e-6,
            beta_initializer=tf.random_normal_initializer(0.0, stddev=0.1),
            gamma_initializer=tf.random_uniform_initializer(0.1, 0.5))
        
        
        # Initialize a list of networks approximating the control alpha_{t_i} at t_i
        self.control_i = []
        
        # Loop over number of time steps
        for j in range(N):
            
            # Batch normalization on dim-dimensional input
            this_control = tf.keras.Sequential() # Sequential groups a linear stack of layers into a tf.keras.Model.
            # Sequential() makes a list of all the layers you want to stack together. Sequential groups a linear stack of layers into a tf.keras.Model.
            this_control.add(tf.keras.layers.Input(dim+j+1)) # tf.keras.Input() is used to instantiate a Keras tensor.
            this_control.add(_bn())
            
            # Two hidden layers of type (Dense -> Batch Normalization -> ReLU)
            for _ in range(2):
                this_control.add(_dense(dim+10+j))
                this_control.add(_bn())
                this_control.add(tf.keras.layers.ReLU()) # adding layer
                
            # Dense layer followed by batch normalization for output
            this_control.add(_dense(1))
            this_control.add(_bn())
            self.control_i.append(this_control)
        
        # The output of this class is a list of networks approximating the control alpha_{t_i} at t_i which is self.control_i.

Next, we define a function to draw multiple realizations of $\tilde C_{N} \approx u(T, X_T)$ by sweeping through the network. The intermediate values $\{\tilde C_{i}\}_{i=0,\ldots,N-1}$ are not stored.
Note that we try to use the network to determine the amount of power at each stage that we store in the battery.

We suppose that we generate power $g_n$ at time $n$, when our current battery has state $b_n$. Then we must choose $\delta b_n := b_{n+1}-b_n$, so that the following all hold:


*   $\delta b_n \in [-b_n, g_n]$, ie we only store energy generated by the turbine
*   $\delta b_n \le \bar{b} - b_n$ ie we cannot exceed the battery capacity.

We can formulate this as: $\delta b_n \in [-b_n, \min\{g_n,(\bar{b}-b_n)\}]$.

In [ ]:
def simulate_Y(inp, model, full_output=False):
    """ This function performs the forward sweep through the network.
    Inputs:
        inp - (price, dG_t, battery_0, max_battery)
        model - model of neural network, contains
            - control_0 - variable approximating the control alpha(0, x)
            - control_i - list of NNs approximating the mapping: x -> alpha_{t_i}
    """
    
    price, dG_t, battery_0, max_battery = inp
    num_sample = price.shape[0] # same dimension as X.shape[0] in DeepBSDE_Solver
    
    N = price.shape[1]-1
    
    p_scale = 100. # Scaling factor for price in calculations
    
    if full_output:
        store_battery = np.zeros(N+1)
        store_grid = np.zeros(N+1)
        store_price = np.zeros(N+1)
        store_cost = np.zeros(N+1)


    #e_num_sample = tf.ones(shape=[num_sample, 1], dtype=DTYPE)      # TensorShape([10, 1])
    e_num_sample = tf.ones(shape=[num_sample,1], dtype=DTYPE)       # TensorShape([10])
    
    # Old Output
    out = 0.05*max_battery*e_num_sample
    
    # y = reward so far
    y = tf.zeros(shape=[num_sample,1], dtype=DTYPE)        # TensorShape([10])

    # Control alpha_{t0} approximation at t0
    z = e_num_sample * model.control_0    # control_0.shape is (1, 100), z.shape is TensorShape([10, 100]).

    # Battery State
    battery_i = tf.ones(shape=[num_sample,1], dtype=DTYPE)  * battery_0     # battery_i.shape is (1, 100), battery_i.shape is TensorShape([10, 100]).   
    #print(battery_i.shape)
    
    # Price process for the cost function. I still need to add to the inputs of simulate_Y
    price = tf.convert_to_tensor(price, dtype=DTYPE)    # This produces TensorShape([10, 100])

    # Power generation process. I still need to add to the inputs of simulate_Y
    power = tf.ones(shape=[num_sample, 1], dtype=DTYPE)  * dG_t  # This produces TensorShape([10, 100])

    if full_output:
        #store_battery[0] = battery_i[0]
        store_price[0] = price[0,0]

    for i in range(N):
        poww = tf.reshape(power[:,i],[num_sample,1])
        pri = tf.reshape(price[:,i],[num_sample,1])
        
        # net battery flow is the amount we send from dG_t into the battery.
        #   Note that this must be between -b_n and min(dG_n, max_b - b_n)
        
        mn = -battery_i
        mx = tf.math.minimum(poww,(max_battery-battery_i))
        
        
        # Select net battery change \delta b_n as f(z)*(mx-mn)/2+ (mx+mn)/2, where
        #.   f(z) -> [-1,1] is increasing, smooth.
        net_battery = 2.*tf.math.atan(z)/(pi) * (mx-mn)/2. + (mx+mn)/2.
        
        battery_i = battery_i + net_battery

        #display(price.shape)
        eta2 = poww - net_battery  #  power[:,i] is TensorShape([10, 100]), while net_battery is TensorShape([10, 100]).
        

        # Compute new value cost value approximations at t_{i+1}. 
        y = y + pri * eta2 - eps * (eta2-out)**2  # y is a tensor with dimension TensorShape([10, 1])
        #         y = y + pri * eta2 - poww * pri # y is a tensor with dimension TensorShape([10, 1])

        out = eta2
        
        z =  tf.reshape(model.control_i[i](tf.concat([p_scale*price[:,:(i+1)],battery_i,out], axis=1)),[num_sample,1])
        
        if full_output:
            store_battery[i] = battery_i[0]
            store_price[i+1] = price[0,i+1]
            store_grid[i] = poww[0] - net_battery[0]
            store_cost[i] = y[0]


    #display(z)
    poww = tf.reshape(power[:,N],[num_sample,1])
    pri = tf.reshape(price[:,N],[num_sample,1])
    mn = -battery_i
    mx = tf.math.minimum(poww,max_battery-battery_i)

    # Select net battery change \delta b_n as f(z)*(mx-mn)/2+ (mx+mn)/2, where
    #.   f(z) -> [-1,1] is increasing, smooth.
    net_battery = 2.*tf.math.atan(z)/(pi) * (mx-mn)/2. + (mx+mn)/2.
    #     net_battery = tf.math.atan(z)/(pi) * max_battery + (max_battery/2 -battery_i)
    battery_i = battery_i + net_battery

    eta2 = poww - net_battery  #  power[:,i] is TensorShape([10, 100]), while net_battery is TensorShape([10, 100]).
    

    # Compute new value cost value approximations at t_{i+1}. 
#     y = y + pri * eta2 - poww * pri  # y is a tensor with dimension TensorShape([10, 1])
    y = y + pri * eta2 - eps*(eta2-out)**2  # y is a tensor with dimension TensorShape([10, 1])


    # Final step
    eta3 = battery_i * 0.95 * pri # average future prices it should be tf.Variable()
    y = y + eta3 

    if full_output:
        store_battery[-1] = battery_i[0]
        store_grid[-1] = poww[0]-net_battery[0]
        store_cost[-1] = y[0]
        return y, store_battery, store_price, store_grid, store_cost
    else:
        return y

Through the application of this function, we may generate for each approximate state path $\{\tilde B_{i}\}_{i=0,\ldots,N}$ with corresponding Brownian motion $\{ W_{t_i}^P \}_{i=0,\ldots,N}$ one realization of $\tilde C_N$ given our unknown network parameters
$$
\theta = \left( \theta_{\alpha_1}, \ldots \theta_{\alpha_{N-1}} \right).
$$


# 4. Evaluation of loss function

In the next step, we define the loss function, i.e., the function to be minimized.

In [ ]:
def loss_fn(inp, model):
    """ This function computes the cost function.
    Inputs:
        inp - (dW_G, dW_P, simOU,dG_t, max_battery)
        model - model of neural network containing C0, control_0, battery_0 and control_i.
    """

    # Forward pass to compute value estimates
    y_pred = simulate_Y(inp, model)
            
    loss = tf.reduce_mean(-y_pred)  # tf.reduce_mean: Computes the mean of elements across dimensions of a tensor.

    return loss

# 5. Computation of the gradient w.r.t. the network parameters $\theta$

The next step uses the automatic differentiation functionality of TensorFlow to compute the gradient of the loss function with respect to the unknowns $\theta$, called trainable_variables in TensorFlow.

In [ ]:
@tf.function
def compute_grad(inp, model):
    """ This function computes the gradient of the loss function w.r.t.
    the trainable variables theta.
    Inputs:
        inp - (dW_G, dW_P, simOU,dG_t, max_battery)
        model - model of neural network containing  C0, control_0, battery_0 and control_i.
    """
    
    with tf.GradientTape() as tape:      # Gradient Tape tracks the automatic differentiation that occurs in a Tensorflow model.
        loss = loss_fn(inp, model)
        
    grad = tape.gradient(loss, model.trainable_variables)
    
    return loss, grad

In [ ]:
# Set learning rate
lr = 1e-2
#lr = 1.

# Choose optimizer for gradient descent step
#optimizer = tf.keras.optimizers.Adam(lr, epsilon=1e-8)
optimizer = tf.keras.optimizers.Adam(lr)

# Initialize neural network architecture
model = BSDEModel()

# Initialize list containing history of losses
history = []

Training without changing the maximum battery.

In [ ]:
t0 = time()

# num_epochs = 40000
# num_epochs = 2000
# num_epochs = 500
num_epochs = 1000

battery_0 = 0.
max_battery = 100
# p0_vec = np.reshape(p0_vec, (k,1))
#p0_vec_new = p0_vec*np.ones((1,num_sample))
#p0_vec = np.array([0.1, 0.2, 0.4, 0.5, 0.7])
#p0_vec = np.reshape(p0_vec,(5,1))
#k=np.size(p0_vec)

# Initialize header of output
print('  Iter          Loss          y   |   Time  Stepsize')

for i in range(num_epochs):
    
    # Each epoch we draw a batch of random paths
    #price = simOU(p0_vec,t_space,noise(num_sample, N, dt)) + cyclical(t_space,10.,0.3,0.5)
    #price = (p0_vec_new,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,dW)
    price_vec = simOU(p0_vec,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,noise(num_sample, N, k, dt))# (5, 101) 5 days
    sum_price_vec_days= np.sum(price_vec, axis=1)
    price_curve = f(t_space) + sum_price_vec_days
    price_curve = np.reshape(price_curve, (num_sample,N+1)) 
    
    # Compute the loss as well as the gradient
    loss, grad = compute_grad((price_curve,dG_t,battery_0,max_battery), model)
    optimizer.apply_gradients(zip(grad, model.trainable_variables))
    
    # Get current Y_0 \approx u(0,x)
#     print(simulate_Y((price,dG_t,battery_0,max_battery), model))
    y = simulate_Y((price_curve[:1,:],dG_t,battery_0,max_battery), model).numpy()[0][0] # It should give the final cost. We are not storing the final cost.

    currtime = time() - t0

    hentry = (i+1, loss.numpy(), y, currtime, lr)
#     print(hentry)
    history.append(hentry)
    if (i+1)%10 == 0:
        print('{:5d}   {:12.4f}   {:8.4f}   | {:6.1f}  {:6.2e}'.format(*hentry))

Plot training history and evolution of approximation of $C_0$.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,6))
xrange = range(len(history))
#ax[0].semilogy(xrange, [e[1] for e in history],'k-')  #  [e[1] for e in history] contains the loss function
ax[0].plot(xrange, [e[1] for e in history]) 
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('training loss')
ax[1].plot(xrange, [e[2] for e in history])    #  [e[2] for e in history] contains the final cost y
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('$C_T$');

In [ ]:
# price = simOU(p0_vec,t_space,noise(num_sample, N, dt)) + cyclical(t_space,10.,0.3,0.5)
#price = simOU(p0_vec,t_space,noise(num_sample, N, dt))

price_vec = simOU(p0_vec,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,noise(2, N, k, dt))
sum_price_vec_days= np.sum(price_vec, axis=1)
price_curve = f(t_space) + sum_price_vec_days
price_curve = np.reshape(price_curve, (2,N+1))
out = simulate_Y((price_curve,dG_t,battery_0,max_battery), model, True)


#price_vec1 = simOU(p0_vec_new,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,noise(num_sample, N, dt)) # (5, 101) 5 days
#sum_price_vec_days1= np.sum(price_vec1, axis=1)
#price_curve1 = f(t_space) + sum_price_vec_days1
#price_curve1 = np.reshape(price_curve1, (N+1,1)) 
#out = simulate_Y((price_curve1,dG_t,battery_0,max_battery), model, True)

bat = out[1]
pri = out[2]+price_shift
gri = out[3]
obj = out[4]+np.cumsum(gri)*price_shift

# price = simOU(p0_vec,t_space,noise(num_sample, N, dt)) + cyclical(t_space,10.,0.3,0.5)
# price = simOU(p0_vec,t_space,noise(num_sample, N, dt))
#price_vec2 = simOU(p0_vec_new,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,noise(num_sample, N, dt)) # (5, 101) 5 days
#sum_price_vec_days2= np.sum(price_vec2, axis=0)
#price_curve2 = f(t_space) + sum_price_vec_days2
#price_curve2 = np.reshape(price_curve2, (N+1,1)) 
#out = simulate_Y((price_curve2,dG_t,battery_0,max_battery), model, True)

price_vec = simOU(p0_vec,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,noise(1, N, k, dt))# (5, 101) 5 days
sum_price_vec_days= np.sum(price_vec, axis=1)
price_curve = f(t_space) + sum_price_vec_days
price_curve = np.reshape(price_curve, (1,N+1))
out = simulate_Y((price_curve,dG_t,battery_0,max_battery), model, True)

bat2 = out[1]
pri2 = out[2]+price_shift
gri2 = out[3]
obj2 = out[4]+np.cumsum(gri2)*price_shift


plt.subplot(2,2,1)
plt.plot(t_space,bat,t_space,bat2)
plt.title("Battery Level")
plt.subplot(2,2,2)
plt.plot(t_space,pri,t_space,pri2)
plt.title("Price")
plt.subplot(2,2,3)
plt.plot(t_space,gri,t_space,gri2)
plt.title("Grid Supply")
plt.subplot(2,2,4)
plt.plot(t_space,obj,t_space,obj2)
plt.title("Value function");

plt.tight_layout()

plt.savefig("7DayTrainedBehaviour.png");

Snapshot

In [ ]:
t1 = int(N/T*2.)

plt.subplot(2,2,1)
plt.plot(t_space[:t1],bat[:t1],t_space[:t1],bat2[:t1])
plt.title("Battery Level")
plt.subplot(2,2,2)
plt.plot(t_space[:t1],pri[:t1],t_space[:t1],pri2[:t1])
plt.title("Price")
plt.subplot(2,2,3)
plt.plot(t_space[:t1],gri[:t1],t_space[:t1],gri2[:t1])
plt.title("Grid Supply")
plt.subplot(2,2,4)
plt.plot(t_space[:t1],obj[:t1],t_space[:t1],obj2[:t1])
plt.title("Cumulative Profit");
plt.tight_layout()

plt.savefig("2DayTrainedBehaviour.png");

Comparing the value function with and without battery.

In [ ]:
plt.plot(t_space,obj,t_space,np.cumsum(dG_t*pri), t_space,obj2, t_space,np.cumsum(dG_t*pri2))
#plt.plot(t_space,np.cumsum(dG_t*pri),t_space,np.cumsum(dG_t*pri2))
plt.title("Total profit over time");
plt.legend(["With Battery_1","No Battery_1", "With Battery_2", "No Battery_2"])

plt.xlabel("Time")
plt.ylabel("Nominal Profit")

plt.tight_layout()

plt.savefig("TrainedVsUntrained.png");

Training for different values of the battery maximum capacity.

In [ ]:
t0 = time()

#num_epochs = 40000
#num_epochs = 2000
#num_epochs = 400
num_epochs = 1000


#battery_0 = 10
battery_0 = 0
#max_battery_vec = [0,5,10,15,20,25,30,35,40]
max_battery_vec = np.linspace(0.,200.,7)

# p0_vec = np.reshape(p0_vec, (5,1))
# #p0_vec_new = p0_vec*np.ones((1,num_sample))
# #p0_vec = np.array([0.1, 0.2, 0.4, 0.5, 0.7])
# #p0_vec = np.reshape(p0_vec,(5,1))
# k=np.size(p0_vec)
expected_profits=np.zeros(np.size(max_battery_vec))
expected_profits_no_battery=np.zeros(np.size(max_battery_vec))

for j in range(len(max_battery_vec)):
    # Initialize header of output
    print('----------------------------------------------------')
    print('Iteration: {:5d}/{:5d}'.format(j+1,len(max_battery_vec)))
    print('----------------------------------------------------')
    print('  Iter          Loss          y   |   Time  Stepsize')

    for i in range(num_epochs):
    
        # Each epoch we draw a batch of random paths
        #price = simOU(p0_vec,t_space,noise(num_sample, N, dt)) + cyclical(t_space,10.,0.3,0.5)
        #price = (p0_vec_new,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,dW)
        price_vec = simOU(p0_vec,t_space,muOU_vec,sigmaOU_vec,thetaOU_vec,noise(num_sample, N, k, dt))# (5, 101) 5 days
        sum_price_vec_days= np.sum(price_vec, axis=1)
        price_curve = f(t_space) + sum_price_vec_days
        price_curve = np.reshape(price_curve, (num_sample,N+1)) 

        # Compute the loss as well as the gradient
        loss, grad = compute_grad((price_curve,dG_t,battery_0,max_battery_vec[j]), model)
        optimizer.apply_gradients(zip(grad, model.trainable_variables))

        # Get current Y_0 \approx u(0,x)
    #     print(simulate_Y((price,dG_t,battery_0,max_battery), model))
        y = simulate_Y((price_curve,dG_t,battery_0,max_battery_vec[j]), model).numpy() # It should give the final cost. We are not storing the final cost.

        z = np.sum(price_curve*dG_t,axis=1)
        
        currtime = time() - t0

        hentry = (i+1, loss.numpy(), np.mean(y), currtime, lr)
    #     print(hentry)
        history.append(hentry)
        if (i+1)%10 == 0:
            print('{:5d}   {:12.4f}   {:8.4f}   | {:6.1f}  {:6.2e}'.format(*hentry))
    expected_profits[j] = np.mean(y)
    expected_profits_no_battery[j] = np.mean(z)
  


Plotting the expected profit for different maximum battery levels.

In [ ]:
excess_profit = expected_profits-expected_profits_no_battery
plt.plot(max_battery_vec, excess_profit,'b^')
# plt.plot(max_battery_vec, expected_profits,'--')
A_lr = np.vstack([max_battery_vec,np.ones(len(max_battery_vec))]).T
m_lr, c_lr = np.linalg.lstsq(A_lr,excess_profit, rcond=None)[0]

plt.plot(max_battery_vec,m_lr*np.array(max_battery_vec) + c_lr,'r--')

plt.xlabel("Maximum battery levels")
plt.ylabel("Expected Excess Profit");
plt.title("Best-fit line of expected excess profit for given battery size")
plt.tight_layout();

plt.savefig("MeanBatteryVsExcessProfit.png")

In [ ]:
plt.plot(max_battery_vec, expected_profits,'b^')
# plt.plot(max_battery_vec, expected_profits,'--')
A_lr = np.vstack([max_battery_vec,np.ones(len(max_battery_vec))]).T
m_lr, c_lr = np.linalg.lstsq(A_lr,expected_profits, rcond=None)[0]

plt.plot(max_battery_vec,m_lr*np.array(max_battery_vec) + c_lr,'r--')

plt.xlabel("Maximum battery levels")
plt.ylabel("Expected Profit");
plt.title("Best-fit line of expected profit for given battery size")
plt.tight_layout();

plt.savefig("MeanBatteryVsProfit.png")

In [ ]:
plt.plot(z,y,'*',z,z,':')

plt.xlabel("Value with no battery")
plt.ylabel("Value with battery")
plt.title("Simulation profit with battery vs no battery");

plt.tight_layout();


plt.savefig("BatteryProfit.png");

print("Mean value with battery: ", np.mean(y))
print("Mean value without battery: ", np.mean(z))
